## Imports:

In [1]:

from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf

from six.moves import cPickle as pickle
from tensorflow.python.framework import ops
import logging

from CIFAR_10.DataGenerator import genTrainValidFolds
from Operations.GraphBuilder import convGraphBuilder, nnGraphBuilder, outputToSoftmax
from Operations.Tools import reshape_data, accuracy
from Operations.Preprocessing import Preprocessing
from Operations.CMNFunctions import lossOptimization



## Reset the Graph every time we initiate a new process

In [2]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

## Create the Network Graph:

In [3]:
class GraphComputer():
    def __init__(self):
        pass
    
    @staticmethod
    def trainGraph(myNet):
        trainData = tf.placeholder(dtype=tf.float32,shape=[None, myNet["imageSize"][0], myNet["imageSize"][1], myNet["numChannels"]],
                                   name='xInputs')
    
        trainLabels = tf.placeholder(dtype=tf.float32, shape=[None, myNet["numLabels"]],
                                     name='yInputs')
        
        runningCount = 1
        
        # Convolutions Layers
        layerOutput = trainData
        for i in np.arange(2):
            # define what layer you need for one stacked convolution Layer
            layers = ["linear", "batchNorm", "nonLinear", "pool", "regularize"]
            layerOutput, _ = convGraphBuilder(xTF=layerOutput,
                                             convKernelSize = myNet["convKernel"][i], convStride = myNet["convStride"][i],
                                             poolKernelSize = myNet["poolKernel"][i], poolStride = myNet["poolStride"][i],
                                             inpDepth = myNet["convDepth"][i], outDepth = myNet["convDepth"][i + 1],
                                             layerNum=runningCount, layers=layers,
                                             axis=[0, 1, 2], isTraining=True)

            runningCount += 1
            
        print ('The shape after convolution layer is : ', layerOutput.get_shape())
        
        # We have to flatten the shape to pass it to the fully connected layer
        # Get the features in flattened fashion
        shapeY, shapeX, depth = layerOutput.get_shape().as_list()[1:4]
        flattenedShape = shapeY * shapeX * depth
        convFeaturesFlattened = tf.reshape(layerOutput, [-1, flattenedShape])
        
        print ('The flattened features of convolutions is: ', convFeaturesFlattened.get_shape())


        # Fully Connected Layers
        myNet["fcLayers"][0] = flattenedShape
        layerOutput = convFeaturesFlattened
        for j in np.arange(2):
            k = i+1+j
            # print (self.myNet["fcLayers"][j])
            layers = ["linear", "batchNorm", "nonLinear", "regularize"]
            layerOutput, _ = nnGraphBuilder(xTF=layerOutput,
                                            numInp = myNet["fcLayers"][j],
                                            numOut = myNet["fcLayers"][j+1],
                                            layerNum = runningCount, layers= layers,
                                            axis=[0], isTraining=True)
            runningCount += 1

        print('The shape after the Fully connected Layer is : ', layerOutput.get_shape())

        # Fully connected to Softmax layer
        outState, probLabel = outputToSoftmax(xTF=layerOutput,
                                              numInp=layerOutput.get_shape().as_list()[1],
                                              numOut = myNet["numLabels"],
                                              layerNum=runningCount)

        print('The shape of the Tensor after Out to Softmax is : ', probLabel.get_shape())
        
        # Loss Function and Optimization
        lossCE, optimizer = lossOptimization(xIN=outState, yIN=trainLabels, 
                                             optimizerParam = myNet["optimizerParam"])
        
        return dict(
                trainData=trainData,
                trainLabels=trainLabels,
                optimizer=optimizer,
                lossCE=lossCE,
                trainPred=probLabel,
        )
    

## Execute the Session:
----------

In [35]:
globalSession = 0

class SesssionExec():
    
    def __init__(self, myNet):
        self.featureDIR = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/2-Class/regularFeatures/RGB/batch_data/"
        self.myNet = myNet


    def runPreprocessor(self, dataIN, sess):
        preprocessedData = np.ndarray(shape=(dataIN.shape), dtype='float32')
        for numImage in np.arange(dataIN.shape[0]):
            feed_dict = {
                self.preprocessGraphDict['imageIN']:dataIN[numImage,:]
            }
            preprocessedData[numImage,:] = sess.run(self.preprocessGraphDict['imageOUT'],
                                                      feed_dict=feed_dict)
        return preprocessedData


    def trainModel(self, dataIN, labelIN, sess):
        batchSize = self.myNet["batchSize"]
        numBatches = int(np.ceil(dataIN.shape[0] / batchSize))
        
        print (dataIN.shape, numBatches, batchSize)
        for numBatch in np.arange(numBatches):
            batchData = dataIN[numBatch * batchSize: (numBatch + 1) * batchSize]
            batchLabels = labelIN[numBatch * batchSize: (numBatch + 1) * batchSize]
            # print('The shape for Batch Data, Batch Labels is: ', batchData.shape, batchLabels.shape)
            # print('The shape for Batch L is: ', batchData.shape)
            feed_dict = {
                self.trainGraphDict['trainData']: batchData,
                self.trainGraphDict['trainLabels']: batchLabels
            }

            _, loss, tpred = sess.run([self.trainGraphDict['optimizer'],
                                       self.trainGraphDict['lossCE'],
                                       self.trainGraphDict['trainPred']],
                                      feed_dict=feed_dict)
            
            convWeightL1 = [v for v in tf.trainable_variables() if v.name == 'Layer1/convWeight:0'][0]
            print ("The shape of Conv layer 1 filter is : ", convWeightL1.get_shape().as_list())
            print (sess.run(convWeightL1))
            print ('00000000000000000000000000000000000000000000000000000000000000000000000000000000')

            if ((numBatch + 1) % 50 == 0) or ((numBatch + 1) == numBatches):
                tacc = accuracy(tpred, batchLabels)
                print("Fold: " + str(self.foldNUM + 1) +
                      ", Epoch: " + str(self.epoch + 1) +
                      ", Mini Batch: " + str(numBatch + 1) +
                      ", Loss= " + "{:.6f}".format(loss) +
                      ", Training Accuracy= " + "{:.5f}".format(tacc))

        return loss, tpred

        
    def execute(self):
        meanValidAcc = 0
        for foldNUM, (trainDataIN, trainLabelsIN,
                      validDataIN, validLabelsIN, labelDict
                      ) in enumerate(
                genTrainValidFolds(self.featureDIR, oneHot=True)):
            
            self.foldNUM = foldNUM
            print('')
            print('#################################################################################')
            trainDataIN, _ = reshape_data(trainDataIN,
                                          imageSize=self.myNet["imageSize"][0],
                                          numChannels=self.myNet["numChannels"])
            
            validDataIN, _ = reshape_data(validDataIN,
                                          imageSize=self.myNet["imageSize"][0],
                                          numChannels=self.myNet["numChannels"])
            
            print('')
            print('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print('The Label Dictionary is given as: ', labelDict)
            print('')
            
            
            # First we reset the graph, so that all the existing graph are erased form memory
            reset_graph()
            
            # Step 1: First we create the Pre-processing Graph:
            self.preprocessGraphDict = Preprocessing().preprocessImageGraph(
                                                            imageSize=self.myNet["imageSize"],
                                                            numChannels=self.myNet["numChannels"])
            
            # Now we create the Training Graph
            self.trainGraphDict = GraphComputer.trainGraph(self.myNet)
            
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                print (sess)
                for epoch in range(self.myNet["epochs"]):
                    self.epoch = epoch
                    # print([op for op in tf.get_default_graph().get_operations()])
                    preprocessedTrainData = self.runPreprocessor(dataIN=trainDataIN, sess=sess)
                    
                    print ('################## ', preprocessedTrainData.shape)
    
                    loss, tpred = self.trainModel(dataIN=trainDataIN,
                                                  labelIN=trainLabelsIN,
                                                  sess=sess)
                    convWeightL1 = [v for v in tf.trainable_variables() if v.name == 'Layer1/convWeight:0'][0]
                    print ("The shape of Conv layer 1 filter is : ", convWeightL1.get_shape().as_list())
                    print (sess.run(convWeightL1))
                    print ('1111111111111111111111111111111111111111111111111111111111111111111111')
                    
                    break
            break
            
#         convWeightL1 = [v for v in tf.trainable_variables() if v.name == 'Layer1/convWeight:0'][0]
#         sess.run(convWeightL1)
            
            

## Define your Network Condiguration and Execute the Computation Graph:
-------------

In [36]:

myNet = dict(imageSize=(32,32),
             numLabels=2,
             numChannels=3,
             convKernel=[(5,5), (5,5)],
             convDepth=[3,64,64],       # The first value of the array should equal to the numChannels
             convStride=[1,1],
             poolKernel=[(2,2), (2,2)],
             poolStride=[1,1],
             fcLayers=[0, 1024, 1024],  # The first value of the array should always be zero because it is updated in
             #  run time
             optimizerParam=dict(optimizer='RMSPROP', learning_rate=0.0001, momentum=0.9),
             batchSize=128,
             epochs = 30)

SesssionExec(myNet).execute()


##########################################################################################

Validation Data and Labels shape:  (1000, 32, 32, 3) (1000, 2)
Training Data and Labels shape:  (9000, 32, 32, 3) (9000, 2)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}

(5, 5, 3, 64)
(64,)
(?, 32, 32, 3)
(5, 5, 64, 64)
(64,)
(?, 32, 32, 64)
The shape after convolution layer is :  (?, 32, 32, 64)
The flattened features of convolutions is:  (?, 65536)
The shape after the Fully connected Layer is :  (?, 1024)
The shape of the Tensor after Out to Softmax is :  (?, 2)
##################  (9000, 32, 32, 3)
(9000, 32, 32, 3) 71 128
The shape of Conv layer 1 filter is :  [5, 5, 3, 64]
[[[[ -2.58773006e-02   3.68441939e-02  -2.20060144e-02 ...,
     -1.01263970e-01   4.22997288e-02  -1.15795068e-01]
   [ -5.56128360e-02  -3.05882208e-02   1.15090273e-01 ...,
     -5.03785200e-02   2.23201048e-02   6.28073737e-02]
   [  6.76007718e-02   3.38114053e-02   2.

KeyboardInterrupt: 

## Important Discussion: 

In [32]:
# We can see the names of all the trained variable taking part in the process by doing the below.
var = [v.name for v in tf.trainable_variables()]
var

['Layer1/convWeight:0',
 'Layer1/convBias:0',
 'Layer1/beta:0',
 'Layer1/gamma:0',
 'Layer2/convWeight:0',
 'Layer2/convBias:0',
 'Layer2/beta:0',
 'Layer2/gamma:0',
 'Layer3/weight:0',
 'Layer3/bias:0',
 'Layer3/beta:0',
 'Layer3/gamma:0',
 'Layer4/weight:0',
 'Layer4/bias:0',
 'Layer4/beta:0',
 'Layer4/gamma:0',
 'Layer5/weight:0',
 'Layer5/bias:0']

In [33]:
# The most recent variable at a particular layer can be fetched as.
with tf.Session() as globalSession:
    globalSession.run(tf.global_variables_initializer())
    convWeightL1 = [v for v in tf.trainable_variables() if v.name == 'Layer1/convWeight:0'][0]
    print (convWeightL1.get_shape().as_list())
    print (globalSession.run(convWeightL1))

[5, 5, 3, 64]
[[[[ -2.58764178e-02   3.68417874e-02  -2.20113825e-02 ...,
     -1.01264648e-01   4.23011743e-02  -1.15798414e-01]
   [ -5.56089245e-02  -3.05920672e-02   1.15085237e-01 ...,
     -5.03812805e-02   2.23189108e-02   6.28041029e-02]
   [  6.76066354e-02   3.38082202e-02   2.55782204e-03 ...,
     -5.41199036e-02  -6.91338331e-02   2.44802367e-02]]

  [[  2.34650761e-01  -2.26471182e-02   1.27404332e-01 ...,
      3.84634919e-02  -6.07750714e-02   9.02874693e-02]
   [  1.18034676e-01   8.27012062e-02   1.28456101e-01 ...,
      8.34089331e-03  -1.41746387e-01   2.87150275e-02]
   [  6.03144430e-02  -7.77993500e-02   1.92963928e-02 ...,
     -2.81373002e-02   3.97584438e-02   2.18043067e-02]]

  [[  7.11000487e-02   2.14320302e-01   1.72619876e-02 ...,
      5.06132245e-02  -1.19313799e-01  -1.43738342e-02]
   [ -1.29597425e-01  -6.56865016e-02   1.50291443e-01 ...,
     -2.00602338e-01   3.73691204e-03  -2.28491098e-01]
   [  1.00871108e-01   1.46426558e-01   9.06365290e-02

In [34]:
with tf.Session() as globalSession1:
    globalSession1.run(tf.global_variables_initializer())
    convWeightL1 = [v for v in tf.trainable_variables() if v.name == 'Layer1/convWeight:0'][0]
    print (convWeightL1.get_shape().as_list())
    print (globalSession1.run(convWeightL1))

[5, 5, 3, 64]
[[[[ -2.58764178e-02   3.68417874e-02  -2.20113825e-02 ...,
     -1.01264648e-01   4.23011743e-02  -1.15798414e-01]
   [ -5.56089245e-02  -3.05920672e-02   1.15085237e-01 ...,
     -5.03812805e-02   2.23189108e-02   6.28041029e-02]
   [  6.76066354e-02   3.38082202e-02   2.55782204e-03 ...,
     -5.41199036e-02  -6.91338331e-02   2.44802367e-02]]

  [[  2.34650761e-01  -2.26471182e-02   1.27404332e-01 ...,
      3.84634919e-02  -6.07750714e-02   9.02874693e-02]
   [  1.18034676e-01   8.27012062e-02   1.28456101e-01 ...,
      8.34089331e-03  -1.41746387e-01   2.87150275e-02]
   [  6.03144430e-02  -7.77993500e-02   1.92963928e-02 ...,
     -2.81373002e-02   3.97584438e-02   2.18043067e-02]]

  [[  7.11000487e-02   2.14320302e-01   1.72619876e-02 ...,
      5.06132245e-02  -1.19313799e-01  -1.43738342e-02]
   [ -1.29597425e-01  -6.56865016e-02   1.50291443e-01 ...,
     -2.00602338e-01   3.73691204e-03  -2.28491098e-01]
   [  1.00871108e-01   1.46426558e-01   9.06365290e-02

In [39]:
class ABC():
    def __init__(self):
        print ("abcdef")
        self.a = 11
    
    @staticmethod
    def printVal(x):
        print (x)
        
ABC.printVal(x=10)

10


NameError: name 'self' is not defined